In [1]:
import pandas as pd
import datetime

In [2]:
pd.options.display.max_columns=500
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [3]:
result = []
render = False

In [4]:
def print_result(df):
    
    df = df.copy()
    
    for i in range(10, len(df), 10):
        print(f"episode {i} average reward: {df[i-10:i].mean()['reward']} max reward: {df[i-10:i].max()['reward']}")

### Global Varaiable & Property

In [5]:
# global variable


agent = None
target_epsilon = None
savefile = None


# global property

render = False
show_action = False

### CODE

In [6]:
import gym

import random
import collections

import time
import threading

from dqn import *


def main():

    parameter = HyperParameter(
        batch_size=32,
        buffer_limit=50000,
        gamma=0.98,
        learning_rate=0.1
    )
    
    global agent
    global epsilon
    global render
    
    env = Environment()
    agent = DQNAgent(param=parameter, path=savefile)

    print_interval = 20
    score = 0.0

    for n_epi in range(100000):
        epsilon = max(0.01, 0.1 - 0.1 * (n_epi / 10000))
        
        if target_epsilon:
            epsilon = target_epsilon
        
        state = env.reset()
        done = False
        
        action_list = []
        episode_reward = 0
        
        while not done:
            action = agent.predict(state, epsilon)
            state_prime, reward, done, info = agent.step(env, state, action)

            state = state_prime
            
            score += reward   
            episode_reward += reward
            
            if render:
                env.render()
                import time
                time.sleep(0.01)
            
            if show_action:
                action_list.append(action)
        
        
        if show_action:
            print(action_list)
            action_list = []
        
        result.append({
            'reward': episode_reward,
            'epsilon': epsilon
        })
        
        if n_epi % print_interval == 0 and n_epi != 0:
            agent.update_network()

            print("n_episode :{}, score : {:.1f}, eps : {:.1f}%".format(
                n_epi, score / print_interval, epsilon * 100))
            score = 0.0

threading.Thread(target=main).start()            
#main()

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
df = pd.DataFrame(result)
df

,reward,epsilon
0,-1814.666667,0.10000
1,-1332.333333,0.09999
2,-696.666667,0.09998
3,-1272.333333,0.09997
4,-1411.333333,0.09996
5,-1244.333333,0.09995
6,-894.333333,0.09994
7,-1186.000000,0.09993
8,567.666667,0.09992
9,-275.666667,0.09991


n_episode :3580, score : -888.4, eps : 6.4%


In [8]:
print_result(df)

In [31]:
render = True

C:\Users\sinny\anaconda3\envs\rl-learn\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


In [10]:
raise Exception

Exception: 

[20220517 084300]
유령 먹는 점수가 eat 점수에 비해 커가지고 agent가 야비하게 아이템을 먹고 쳐 기다린다... 보상점수를 좀 줄여야 할 듯 하다.
결국엔 전체적인 게임 흐름이 아니라 독립되어 한 상태에서 보상 값읅 구하는거기때문에 유령 먹는점수를 하향하는게 매우 중요하다
왜 계속 구석에 가있나 햇네 ㅋㅋㅋ


ghost_reward 변수를 새롭게 정의후 200에서 0으로 수정 유령은 필수로 먹어야하는 요소가 아니며 살아남기 위한 요소라거 정의하도록 한다.